Install Libraries

In [5]:
!pip install gradio langchain langchain_groq langchain_community faiss-cpu Pillow pytesseract pdf2image PyPDF2 sentence_transformers groq tiktoken
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

Mount Google Drive

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

drive_base = "/content/drive/MyDrive/BioChatbot"
pdf_folder = os.path.join(drive_base, "pdfs")
img_folder = os.path.join(drive_base, "pdf_images")
os.makedirs(pdf_folder, exist_ok=True)
os.makedirs(img_folder, exist_ok=True)

print(f" Data will be saved in Google Drive under: {drive_base}")

Mounted at /content/drive
 Data will be saved in Google Drive under: /content/drive/MyDrive/BioChatbot


Upload PDFs to Drive

In [2]:
from google.colab import files

uploaded = files.upload()

for filename, content in uploaded.items():
    # Destination in Drive
    destination_path = os.path.join(pdf_folder, filename)
    with open(destination_path, "wb") as f:
        f.write(content)
    print(f" Saved to Drive: {destination_path}")

print(" All PDFs saved permanently in Google Drive.")

 All PDFs saved permanently in Google Drive.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OCR + Save Images

In [6]:
from pdf2image import convert_from_path
import pytesseract
from PyPDF2 import PdfReader
from multiprocessing import Pool, cpu_count

def process_page(args):
    pdf_path, filename, page_num = args
    try:
        # convert PDF page to image
        image = convert_from_path(pdf_path, dpi=150, first_page=page_num, last_page=page_num)[0]

        # save page image in drive
        image_path = os.path.join(img_folder, f"{filename}_page_{page_num}.png")
        image.save(image_path, "PNG")

        # OCR extract text
        text = pytesseract.image_to_string(image)
        return f"\n\n--- Page {page_num} of {filename} ---\n{text}"
    except Exception as e:
        return f"\n\n--- Page {page_num} of {filename} ---\nERROR: {e}"

def ocr_pdfs_in_drive(pdf_folder, output_file):
    tasks = []
    for filename in sorted(os.listdir(pdf_folder)):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            try:
                reader = PdfReader(pdf_path)
                total_pages = len(reader.pages)
                print(f" {filename} → {total_pages} pages")
                for page_num in range(1, total_pages + 1):
                    tasks.append((pdf_path, filename, page_num))
            except Exception as e:
                print(f" Could not read {filename}: {e}")

    with Pool(cpu_count()) as pool:
        for result in pool.imap_unordered(process_page, tasks):
            with open(output_file, "a", encoding="utf-8") as f:
                f.write(result)

    print(f" OCR completed. Output saved to {output_file}")

ocr_text_file = os.path.join(drive_base, "ocr_output.txt")
ocr_pdfs_in_drive(pdf_folder, ocr_text_file)

 Unit 01-Introduction to Biology-English.pdf → 5 pages
 Unit 02-Chemical and cellular basis of life-English.pdf → 66 pages
 Unit 03-Evolution and diversity of organisms-English.pdf → 35 pages
 Unit 04-Plant form and function-English.pdf → 49 pages
 Unit 05 Part 1-Animal form and function-English.pdf → 80 pages
 Unit 05 Part 2-Animal form and function-English.pdf → 88 pages
 Unit 06-Genetics-English.pdf → 43 pages
 Unit 07-Molecular Biology and Recombinant DNA Technology-English.pdf → 61 pages
 Unit 08-Environmental Biology-English.pdf → 43 pages
 Unit 09-Microbiology-English.pdf → 37 pages
 Unit 10-Applied Biology-English.pdf → 30 pages
 OCR completed. Output saved to /content/drive/MyDrive/BioChatbot/ocr_output.txt


Load OCR Text

In [7]:
with open(ocr_text_file, "r", encoding="utf-8") as f:
    ocr_text = f.read()

full_text_path = os.path.join(drive_base, "biology_full_content.txt")
with open(full_text_path, "w", encoding="utf-8") as f:
    f.write(ocr_text)

print(" Text saved in Google Drive.")

 Text saved in Google Drive.


Create FAISS Vector DB

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.create_documents([ocr_text])

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)

# save FAISS index to drive
vectorstore.save_local(os.path.join(drive_base, "faiss_index"))

print(" FAISS index saved in Google Drive.")

/tmp/ipython-input-590293367.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 FAISS index saved in Google Drive.


Load Groq LLM

In [9]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

os.environ["GROQ_API_KEY"] = "API_Key"
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.2)

qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

print(" Biology Chatbot Ready!")


 Biology Chatbot Ready!


Gradio Chat Interface

In [11]:
import gradio as gr

def chat_with_bot(user_input, history):
    try:
        answer = qa.run(user_input)
        return answer
    except Exception as e:
        return f" Error: {str(e)}"

demo = gr.ChatInterface(
    fn=chat_with_bot,
    title="A/L Biology Chatbot",
    description="Ask any A/L Biology Questions.",
    theme="default",
)

demo.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33f9bbefa8d176a3bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
